In [6]:
import openmeteo_requests
import requests_cache
import time
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define the API endpoint and parameters
url = "https://archive-api.open-meteo.com/v1/archive"
start_date = "2020-01-01"
end_date = "2024-05-31"
daily_variables = ["temperature_2m_max", "daylight_duration"]

# Read input CSV file
input_file = 'coordinates.csv'
output_file = 'updated_coordinates.csv'

# Read coordinates from CSV and process each one
coordinates_df = pd.read_csv(input_file)

# Initialize lists to hold new data
all_temperature_2m_max = []
all_daylight_duration = []

for index, row in coordinates_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily_variables
    }
    
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    
    # Extract and print some meta information
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+2 {response.UtcOffsetSeconds()} s")
    
    # Process daily data
    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(1).ValuesAsNumpy()
    
    # Calculate the average values for the given date range
    avg_temp_max = daily_temperature_2m_max.mean() if len(daily_temperature_2m_max) > 0 else None
    avg_daylight = daily_daylight_duration.mean() if len(daily_daylight_duration) > 0 else None
    
    # Append to lists
    all_temperature_2m_max.append(avg_temp_max)
    all_daylight_duration.append(avg_daylight)
    
    # To avoid hitting the API rate limit
    time.sleep(1)  # Sleep for 1 second between requests

# Add new data as columns to the original DataFrame
coordinates_df['avg_temperature_2m_max'] = all_temperature_2m_max
coordinates_df['avg_daylight_duration'] = all_daylight_duration

# Save the updated DataFrame to a new CSV file
coordinates_df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")


Coordinates 29.490333557128906°N 35.06358337402344°E
Elevation 411.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 31.739892959594727°N 35.87034606933594°E
Elevation 769.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 32.302284240722656°N 36.21818161010742°E
Elevation 680.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 32.513179779052734°N 36.02190017700195°E
Elevation 504.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 30.193321228027344°N 35.78947448730469°E
Elevation 1099.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 32.302284240722656°N 36.0°E
Elevation 634.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 30.75571060180664°N 35.70248031616211°E
Elevation 1342.0 m asl
Timezone None None
Timezone difference to GMT+2 0 s
Coordinates 32.302284240722656°N 35.890907287597656°E
Elevation 1016.0 m asl
Timezone None None
Timezone difference to GMT+

OpenMeteoRequestsError: {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}